# Лекция №2

# Гистограммы 

Так что такое гистограмма? Вы можете рассматривать гистограмму как график, который дает вам общее представление о распределении интенсивности изображения. Это график со значениями пикселей (от $0$ до $255$, не всегда) по оси $X$ и соответствующим количеством пикселей в изображении по оси $Y$.

Это просто еще один способ понять образ. Глядя на гистограмму изображения, вы получаете представление о контрасте, яркости, распределении интенсивности и т.д. этого изображения. Почти все инструменты обработки изображений сегодня предоставляют функции на гистограмме.

<img src="https://i.ibb.co/cJxrWhx/histogram_sample.jpg" alt="Drawing" style="width: 400px;"/>

Вы можете увидеть изображение и его гистограмму. (Помните, эта гистограмма нарисована для изображения в оттенках серого, а не для цветного изображения). Левая область гистограммы показывает количество более темных пикселей на изображении, а правая область показывает количество более ярких пикселей. На гистограмме видно, что темная область больше, чем яркая, а количество полутонов (значения пикселей в среднем диапазоне, скажем, около $127$) много меньше.

**Гистограммы** $-$ это собранные данные, организованные в набор предопределенных столбцов.
Когда мы говорим данные, мы не ограничиваем их значениями интенсивности (как мы видели в предыдущем уроке). Собранные данные могут быть любой функцией, которую вы найдете полезной для описания вашего изображения.

Давайте посмотрим на пример. Представьте, что матрица содержит информацию об изображении (то есть интенсивность в диапазоне $0-255$):

<img src="img/Histogram_Calculation_Theory_Hist.jpg" alt="Drawing" style="width: 400px;"/>

Что произойдет, если мы захотим считать эти данные организованным способом? Так как мы знаем, что диапазон информационных значений для этого случая равен 256 значениям, мы можем сегментировать наш диапазон по частям (называемым ячейками), например:

$$
{\begin{array}{l}
[0, 255] = { [0, 15] \cup [16, 31] \cup ....\cup [240,255] } \\
range = { bin_{1} \cup bin_{2} \cup ....\cup bin_{n = 15} }
\end{array}}
$$

и мы можем вести подсчет количества пикселей, попадающих в диапазон каждого $bin_{i}$. Применяя это к примеру выше, мы получаем изображение ниже (ось x представляет ячейки, а ось y – количество пикселей в каждом из них).

<img src="https://i.ibb.co/Wss2Fqh/Histogram_Calculation_Theory_Hist1.jpg" alt="Drawing" style="width: 400px;"/>

Это был простой пример того, как работает гистограмма и почему она полезна. Гистограмма может вести учет не только интенсивности цвета, но и любых характеристик изображения, которые мы хотим измерить (то есть градиенты, направления и т.д.).

Давайте выделим некоторые части гистограммы:

* __dims__: количество параметров, для которых вы хотите собрать данные. В нашем примере dims = $1$, потому что мы рассчитываем только значения интенсивности каждого пикселя (в полутоновом изображении)
* __bins__: это количество подразделений в каждом тусклом свете. В нашем примере bins = $16$
* __range__: пределы измеряемых значений. В этом случае: range = $[0,255]$

Что если вы хотите сосчитать две особенности? В этом случае ваша результирующая гистограмма будет трехмерным графиком (в котором $x$ и $y$ будут $bin_{x}$ и $bin_{y}$ для каждого объекта, а $z$ будет количеством отсчетов для каждой комбинации $(bin_ {x}, bin_ {y})$. То же самое относится и к дополнительным функциям (конечно, это становится сложнее).

In [ ]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt

%matplotlib inline

### Гистограммы в OpenCV

Воспользуемся функцией ```cv2.calcHist()```, чтобы найти гистограмму. Давайте ознакомимся с функцией и ее параметрами:

```cv2.calcHist (image, channel, mask, histSize, range , hist, accumulate)```

* **images** $-$ это исходное изображение типа $uint8$ или $float32$. его следует указывать в квадратных скобках, т.е. **[img]**.
* **channel** $-$ также указывается в квадратных скобках. Это индекс канала, для которого мы рассчитываем гистограмму. Например, если входное изображение представляет собой изображение в градациях серого, его значение равно $[0]$. Для цветного изображения вы можете передать $[0]$, $[1]$ или $[2]$, чтобы вычислить гистограмму синего, зеленого или красного канала соответственно.
* **mask** $-$ маска изображения. Чтобы найти гистограмму полного изображения, она задается как **None**. Но если вы хотите найти гистограмму определенной области изображения, вы должны создать для нее изображение маски и указать ее как маску.
* **histSize** $-$ это представляет наш счетчик __bin__. Нужно указывать в квадратных скобках. Для полной шкалы мы передаем $[256]$.
* **range** $-$ это наш **range**. Обычно это $[0,256]$.
* **hist** $-$ выходная гистограмма, представляющая собой плотный или разреженный размерный массив.
* **accumulate** $-$ накопительный флаг. Если он установлен, гистограмма не очищается в начале при выделении. Эта функция позволяет вычислять одну гистограмму из нескольких наборов массивов или своевременно обновлять гистограмму.

Итак, начнем с примера изображения. Просто загрузите изображение в режиме оттенков серого и найдите его полную гистограмму.

In [ ]:
image = cv2.imread('img/forest.jpg')
h, w, _ = image.shape
image = cv2.resize(image.copy(), (w // 2, h // 2))
image = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)

# grayscale
gray = cv2.cvtColor(image.copy(), cv2.COLOR_RGB2GRAY)

# получим гистограмму
# ravel() - разворачивает массив в список
hist = cv2.calcHist([gray.ravel()], [0], None, [256], [0,256])

fig, m_axs = plt.subplots(1, 2, figsize=(12,4))
ax1, ax2 = m_axs

ax1.set_title('Исходное изображение')
ax1.imshow(image)
ax2.set_title('Гистограмма по освещенности')
ax2.plot(hist)
ax2.set_xlim([0,256]);

### Гистограммы в  Numpy

```Numpy``` также предоставляет вам функцию ```np.histogram()```.

In [ ]:
hist, bins = np.histogram(gray.ravel(), 256, [0,256])

fig, m_axs = plt.subplots(1, 2, figsize=(12,4))
ax1, ax2 = m_axs

ax1.set_title('Исходное изображение')
ax1.imshow(image)
ax2.set_title('Гистограмма по освещенности')
ax2.plot(hist)
ax2.set_xlim([0,256]);

Гистограмма аналогичная, как мы рассчитывали ранее. Но ячейки будут иметь $257$ элементов, потому что **Numpy** рассчитывает ячейки как $0-0.99$, $1-1.99$, $2-2.99$ и т.д. Таким образом, конечный диапазон будет $255-255.99$. Чтобы представить это, они также добавляют $256$ в конце бинов. Но нам не нужно это $256$. До $255$ достаточно.

Визуализируем сразу $3$ канала цвета:

In [ ]:
fig, m_axs = plt.subplots(1, 2, figsize=(12,4))
ax1, ax2 = m_axs

ax1.set_title('Исходное изображение')
ax1.imshow(image)
ax2.set_title('Гистограмма по освещенности')
ax2.set_xlim([0,256])

color = ('r', 'g', 'b')
for i, col in enumerate(color):
    hist, bins = np.histogram(image[..., i].ravel(), 256, [0, 256])
    ax2.plot(hist, color=col)

### Применение маски

Мы использовали ```cv2.calcHist()```, чтобы найти гистограмму полного изображения. Что если вы хотите найти гистограммы некоторых областей изображения? Просто создайте изображение маски белым цветом на области, где вы хотите найти гистограмму, и черным в противном случае. Затем передайте это как маску.

In [ ]:
# grayscale
gray = cv2.cvtColor(image.copy(), cv2.COLOR_RGB2GRAY)

# создадим маску c помощью GUI ROI
# маска должна быть одномерной
mask = np.zeros(image.shape[:2], np.uint8)

mask[50:, 600:850] = 1

masked_img = cv2.bitwise_and(gray, gray, mask=mask)

## проверим, что мы вырезали то, что хотели
plt.title('Маска')
plt.imshow(masked_img, cmap='gray');

Тут мы воспользовались распространенным приемом с *"маской"*, т.е. использовали массив из нулей, на которой наложили интересующую картнику. 

Для арифметических операция над матрицами в OpenCV есть несклько базовых функций. Рассмотрим несколько из них:

* __cv2.absdiff(src1, src2[,dst])__

    абсолютная разница между двумя массивами: $${\texttt{dst}(I)=|\texttt{src1}(I)-\texttt{src2}(I)|}$$

* __cv2.add(src1, src2[, dst[, mask[, dtype]]])__

    Функция add рассчитывае сумму двух массивов: $${\texttt{dst}(I)=\texttt{src1}(I)+\texttt{src2}(I) \quad\texttt{if mask}(I)\ne0}$$

* __cv2.bitwise_and(src1, src2[, dst[, mask]])__

    Функция рассчитывает побитовое логическое соединение для каждого элемента массива: $${\texttt{dst} (I)=\texttt{src1}(I)\wedge\texttt{src2}(I)\quad\texttt{if mask}(I)\ne0}$$

* __cv2.bitwise_not(src[, dst[, mask]])__

    Функция рассчитывает побитовую инверсию входного массива для каждого элемента: $${\texttt{dst}(I)=\neg{\texttt{src}(I)}}$$

* __cv2.bitwise_or(src1, src2[, dst[, mask]])__

    Функция вычисляет побитовую логическую дизъюнкцию для каждого элемента для: $${\texttt{dst}(I)=\texttt{src1}(I)\vee\texttt{src2}(I)\quad\texttt{if mask}(I)\ne0}$$

* __cv2.bitwise_xor(src1, src2[, dst[, mask]])__

    Функция вычисляет побитовую логическую операцию «исключая или» для каждого элемента массива: $${\texttt{dst}(I)=\texttt{src1}(I)\oplus\texttt{src2}(I)\quad\texttt{if mask}(I)\ne0}$$

Остальные можно найти в [документации](https://docs.opencv.org/3.4.2/d2/de8/group__core__array.html)

In [ ]:
## посчитаем гистограмму для всего изображения и маски
hist_full = cv2.calcHist([gray],[0],None,[256],[0,256])
hist_mask = cv2.calcHist([gray],[0],mask,[256],[0,256])


fig, m_axs = plt.subplots(2, 2, figsize=(12,8))
ax1, ax2 = m_axs

ax1[0].set_title('Исходное изображение')
ax1[0].imshow(gray, cmap='gray')
ax1[1].set_title('Маска')
ax1[1].imshow(masked_img, cmap='gray')
ax2[0].set_title('Освещенность изображения')
ax2[0].plot(hist_full)
ax2[0].set_xlim(0, 256)
ax2[1].set_title('Освещенность маски')
ax2[1].plot(hist_mask)
ax2[1].set_xlim(0, 256);

Теперь посмотрим для всех каналов цвета интесивность на маске

In [ ]:
masked_img = cv2.bitwise_and(image, image, mask=mask)

fig, m_axs = plt.subplots(2, 2, figsize=(12,8))
ax1, ax2 = m_axs

ax1[0].set_title('Исходное изображение')
ax1[0].imshow(image, cmap='gray')
ax1[1].set_title('Маска')
ax1[1].imshow(masked_img, cmap='gray')
ax2[0].set_title('Освещенность изображения')
ax2[1].set_title('Освещенность маски')

color = ('r', 'g', 'b')
for i,col in enumerate(color):
    hist_full = cv2.calcHist([image], [i], None, [256], [0,256])
    hist_mask = cv2.calcHist([image], [i], mask, [256], [0,256])
    ax2[0].plot(hist_full, color=col)
    ax2[0].set_xlim(0, 256)
    ax2[1].plot(hist_mask, color=col)
    ax2[1].set_xlim(0, 256)

## Выравнивание гистограм

## Функция распределения

Прежде, чем переходить к алгоритмам выравнивания гистограмм, рассмотрим такую функцию, как __функция распределения__. 

Это понятие пошло из математической статистики. Обычно говорят функция распределения случайной величины. В каком-то смысле значения в каждом пикселе - случаыные величины, поэтому аналогия вполне уместна. Отличие будет в том, что наша функция будет нормирована на максимальное значение из всех случайных величин. В статистике функция распределения нормирована на 1 из-за того, что там говорят о вероятностях.

Согласно определению, дискретная функции распределения случайной величины выглядит следующим образом:


Если случайная величина $X$ дискретна, то есть её распределение однозначно задаётся функцией вероятности $\mathbb {P} (X=x_{i})=p_{i},\;i=1,2,\ldots$,
то функция распределения $F_{X}$ этой случайной величины кусочно-постоянна и может быть записана как:

$F_{X}(x)=\sum \limits _{i\colon x_{i}\leqslant x}p_{i}$.

_Рассмотрим следующий пример:_

| value | 1 | 2 | 3 |
| --- | --- | --- | --- |
| $x$ | -5 | 2.5 | 10 |
| $p$ | 0.5| 0.4 | 0.1 |

Чему, например, равно значение $F(-20)$? Это вероятность того, что выигрыш будет меньше, чем $–20$. И это невозможное событие: $F(-20)=P(x < -20)=0$. Совершенно понятно, что $F(x)=0$  и для всех $x$ из интервала $(-\infty, -5)$, а также для $x=-5$. Почему? По определению функции распределения:

$F(-5) = p(x < -5) = 0$ – вы согласны?  Функция $F(x)$ возвращает вероятность того, что в точке $x=-5$ вероятность будет СТРОГО МЕНЬШЕ «минус» пяти.

Таким образом: $F(x)=0$, если $x\le -5$.

На интервале $-5<x<2.5$ функция $F(x)=p(X<x)=0.5$, поскольку левее любой точки этого интервала есть только одно значение  $x_1 =-5$ случайной величины, которое появляется с вероятностью $0.5$. Кроме того, сюда же следует отнести точку $x=2.5$, так как:

F(2.5)=p(x<2.5)=0.5 – очень хорошо осознайте этот момент!

Таким образом, если $-5<x\le 2.5$ , то $F(x)=0.5$

Далее рассматриваем  промежуток $2.5<x\le 10$. СТРОГО ЛЕВЕЕ любой точки этого промежутка находятся две вероятности $x_1=-5$, $x_2=2.5$, поэтому:

$F(x)=p(X<x)=0.5+0.4=0.9$

И, наконец, если $x>10$, то $F(x)=p(X<x)=0.5+0.4+0.1=1$, ибо все значения $x_1=-5$, $x_2=2.5$, $x_3=10$ случайной величины $X$ лежат строго левее любой точки $x\in(10, \infty)$

Заметим, __важную вещь__: коль скоро, функция $F(x)$ характеризуем вероятность, то она может принимать значения лишь из промежутка $0 \le F(x) \le 1$ – и никакие другие!

Итак, функция распределения вероятностей является кусочной и, как многие знают, в таких случаях принято использовать фигурные скобки:

${F(x)=\begin{cases} 0, если\ x\le -5 \\ 0.5, если \ -5 < x \le 2.5 \\ 0.9, если \ 2.5 < x \le 10 \\ 1, если \ x > 10 \end{cases}}$

График данной функции имеет разрывный «ступенчатый» вид:

<img src="img/funkcia_raspredeleniya_dsv.jpg" alt="Drawing" style="width: 400px;"/>

### Вернемся к гистограммам

Рассмотрим изображение, значения пикселей которого ограничены только определенным диапазоном значений. Например, более яркое изображение будет иметь все пиксели, ограниченные высокими значениями. Но хорошее изображение будет иметь пиксели из всех областей изображения. Таким образом, вам нужно растянуть эту гистограмму до конца (как показано на изображении ниже), и это то, что делает выравнивание гистограммы (если коротко). Это обычно улучшает контраст изображения.

<img src="https://i.ibb.co/rZ3sM6q/histogram_equalization.png" alt="Drawing" style="width: 400px;"/>

Я бы порекомендовал вам прочитать страницу Википедии по выравниванию гистограммы для более подробной информации. Он имеет очень хорошее объяснение с отработанными примерами, так что вы сможете понять почти все после прочтения этого. Вместо этого здесь мы увидим его реализацию Numpy. После этого мы увидим функцию OpenCV.

Нам понадобится функция распределения значений (cumulative distribution function) пикслей в градации серого: 

$${cdf(x)=h(0)+h(1)+\dots+h(x),}$$ где $h(x)$ $-$ функция гистограммы

Значение функции распределения показывает, какое количество пикселей имеют яркости из отрезка $[0,x]$.

### Расчитаем функцию распределения

In [ ]:
# гистограмма
hist, bins = np.histogram(image.flatten(), 256, [0,256])
# функция распределения
cdf = hist.cumsum()


fig, m_axs = plt.subplots(1, 2, figsize=(12, 4))
ax1, ax2 = m_axs

ax1.set_title('Гистограмма')
ax1.plot(hist)
ax2.set_title('Функция распределения значений гистограммы')
ax2.plot(cdf)
ax2.set_xlim(0, 256);

In [ ]:
hist, bins = np.histogram(image.flatten(), 256, [0,256])

## фунция распределения
cdf = hist.cumsum()
cdf_normalized = cdf * hist.max() / cdf.max()

fig, m_axs = plt.subplots(1, 2, figsize=(18, 6))
ax1, ax2 = m_axs

ax1.set_title('Исходное изображение')
ax1.imshow(image)
ax2.set_title('Гистограмма по освещенности')
ax2.plot(hist, color='r')
ax2.plot(cdf_normalized, color='b')
ax2.legend(('histogram', 'cdf'), loc = 'upper left')
ax2.set_xlim(0, 256);

Вы можете видеть гистограмму, лежащую в более ярком регионе. Нам нужен полный спектр. Для этого нам нужна функция преобразования, которая отображает входные пиксели в более яркой области для вывода пикселей в полной области. Это то, что делает выравнивание гистограммы.

Теперь мы находим минимальное значение гистограммы (исключая $0$) и применяем уравнение выравнивания гистограммы. Для маскированного массива все операции выполняются на немаскированных элементах. Вы можете прочитать больше об этом в документации Numpy на масках.

In [ ]:
# еще раз глянем что же такое cdf

cdf

In [ ]:
cdf_m = np.ma.masked_equal(cdf.copy(), 0)

# выравнивание к линейному распределению (min/max)
cdf_m = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max() - cdf_m.min())

cdf = np.ma.filled(cdf_m, 0).astype('uint8')

In [ ]:
# применим расчитанное распределение к исходному изображению
image2 = cdf[image.copy()]  

In [ ]:
# разберем, что тут вообще произошло

image[:3, :3], cdf[image[:3, :3]]

In [ ]:
# почему уменно так?

cdf[192], cdf[244]

Теперь у нас есть справочная таблица, которая дает нам информацию о том, что является значением выходного пикселя для каждого значения входного пикселя.

In [ ]:
# снова считаем гистограмму и ее распределение
hist2, bins2 = np.histogram(image2.flatten(), 256, [0,256])
cdf2 = hist2.cumsum()
cdf_normalized2 = cdf2 * hist2.max() / cdf2.max()


fig, m_axs = plt.subplots(1, 2, figsize=(12, 4))
ax1, ax2 = m_axs

ax1.set_title('Исходная гистограмма по освещенности')
ax1.plot(hist, color='r')
ax1.plot(cdf_normalized, color='b')
ax1.legend(('histogram', 'cdf'), loc = 'upper left')
ax1.set_xlim(0, 256)

ax2.set_title('Выравненная гистограмма по освещенности')
ax2.plot(hist2, color='r')
ax2.plot(cdf_normalized2, color='b')
ax2.legend(('histogram', 'cdf'), loc = 'upper left')
ax2.set_xlim(0, 256);

In [ ]:
fig, m_axs = plt.subplots(1, 2, figsize=(24, 8))
ax1, ax2 = m_axs

ax1.set_title('Исходное изображение')
ax1.imshow(image)
ax2.set_title('Выравненное изображение')
ax2.imshow(image2);

OpenCV имеет функцию для этого, ```cv2.equalizeHist()```. Его вход $-$ это просто изображение в градациях серого, а вывод – изображение, выровненное по гистограмме. Однако, реализация в OpenCV рабоает зачастую медленне, чем ```Numpy```.

## Адаптивный эквалайзер с ограниченной адаптивной гистограммой (CLAHE)

Первое выравнивание гистограммы, которое мы только что видели, учитывает глобальный контраст изображения. Во многих случаях это не очень хорошая идея. Например, ниже изображение показывает входное изображение и его результат после глобальной выравнивания гистограммы.

In [ ]:
## вспомним, что было с линиаризацией освещенности
image = cv2.imread('img/forest.jpg')
gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
equ = cv2.equalizeHist(gray)

In [ ]:
fig, m_axs = plt.subplots(1, 2, figsize=(24, 8))
ax1, ax2 = m_axs

ax1.set_title('Исходное изображение')
ax1.imshow(gray, cmap='gray')
ax2.set_title('Выравненное изображение')
ax2.imshow(equ, cmap='gray');

Это правда, что контрастность фона улучшилась после выравнивания гистограммы. Мы потеряли большую часть информации там из-за чрезмерной яркости. Это потому, что его гистограмма не ограничена определенной областью, как мы видели в предыдущих случаях.

Поэтому для решения этой проблемы используется __адаптивное выравнивание гистограммы (AHE)__. При этом изображение делится на маленькие блоки, называемые «плитками» (tileSize по умолчанию в OpenCV составляет $8\times8$). Затем каждый из этих блоков гистограммы выравнивается как обычно. Таким образом, в небольшой области гистограмма будет ограничена небольшой областью (если нет шума), как показано на рисунке:

<img src="https://i.ibb.co/Z2v37jw/AHE-neighbourhoods.png" alt="Drawing" style="width: 300px;"/>

Если шум есть, он будет усилен. Чтобы избежать этого, применяется ограничение контраста. Если какой-либо интервал гистограммы превышает указанный предел контраста (по умолчанию $40$ в OpenCV), эти пиксели обрезаются и распределяются равномерно по другим интервалам перед применением выравнивания гистограммы. После выравнивания для удаления артефактов на границах мозаики применяется билинейная интерполяция.

Ниже приведен фрагмент кода, демонстрирующий применение CLAHE в OpenCV:

In [ ]:
# применим CLAHE из cv2 и сравним результат с обычным варвниваем всей гистрограммы
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl1 = clahe.apply(gray)

hist_gray = cv2.calcHist([gray], [0], None, [256], [0,256])
hist_equ = cv2.calcHist([equ], [0], None, [256], [0,256])
hist_clahe = cv2.calcHist([cl1], [0], None, [256], [0,256])

fig, m_axs = plt.subplots(2, 3, figsize=(24, 12))
ax1, ax2 = m_axs

ax1[0].set_title('ЧБ изображение')
ax1[0].imshow(gray, cmap='gray')
ax1[1].set_title('Выравненная гистограмма')
ax1[1].imshow(equ, cmap='gray')
ax1[2].set_title('CLAHE')
ax1[2].imshow(cl1, cmap='gray')

ax2[0].plot(hist_gray)
ax2[1].plot(hist_equ)
ax2[2].plot(hist_clahe);

## Практика

Воспользуемся реализованными ранее функциями скользящего окна и реализуем собственный CLAHE для каждого канала изображения, а затем сравним нашу рализацию и ```cv2```.

In [ ]:
# Ваш код

## Сравнение гистограмм 

Чуть выше мы уже касались сравнения изображений с некими случайными процессами, точнее значения пикселей мы принимаем в качестве случайно величины. Проводя такую аналогию можно вводить принципы и методы статистического сравнения изображений между собой. Далее рассмотрим некоторые важные величины из математической стаистики, которые пригодятся дальше.

### Матемаматическое ожидание### 
 Мат ожидание означает среднее (взвешенное по вероятностям возможных значений) значение случайной величины.

В англоязычной литературе обозначается через ${\mathbb  {E}}[X]$, в русскоязычной — $M[X]$. В статистике часто используют обозначение $\mu$.

Для случайной величины, принимающей значения только $0$ или $1$ математическое ожидание равно $p$ — вероятности "единицы". Математическое ожидание суммы таких случайных величин равно $np$, где $n$ — количество таких случайных величин.

На практике математическое ожидание обычно оценивается как среднее арифметическое наблюдаемых значений случайной величины (выборочное среднее, среднее по выборке). Доказано, что при соблюдении определенных слабых условий (в частности, если выборка является случайной, то есть наблюдения являются независимыми) выборочное среднее стремится к истинному значению математического ожидания случайной величины при стремлении объема выборки (количества наблюдений, испытаний, измерений) к бесконечности.

_Определение для дискретной случайной величины._

Если $X$ — дискретная случайная величина, имеющая распределение ${\displaystyle \mathbb {P} (X=x_{i})=p_{i},\;\sum \limits _{i=1}^{\infty }p_{i}=1}$, то ${\displaystyle M[X]=\sum \limits _{i=1}^{\infty }x_{i}\,p_{i}}$.

__Свойства__

* Математическое ожидание числа (не случайной, фиксированной величины, константы) есть само число.

    ${\displaystyle M[a]=a}$, где ${\displaystyle a\in \mathbb {R} }$ — константа.
    
    
* Математическое ожидание линейно, то есть

    ${\displaystyle M[aX+bY]=aM[X]+bM[Y]}$,
    где ${\displaystyle X,Y}$ — случайные величины с конечным математическим ожиданием, а ${\displaystyle a,b\in \mathbb {R} }$ — произвольные константы.
   
   В частности, математическое ожидание суммы (разности) случайных величин равно сумме (соответственно - разности) их математических ожиданий.


* Математическое ожидание сохраняет неравенства, то есть если ${\displaystyle 0\leqslant X\leqslant Y}$ почти наверняка, и ${\displaystyle Y}$ — случайная величина с конечным математическим ожиданием, то математическое ожидание случайной величины ${\displaystyle X}$ также конечно, и более того

    ${\displaystyle 0\leqslant M[X]\leqslant M[Y]}$
    

* Математическое ожидание не зависит от поведения случайной величины на событии вероятности нуль, то есть если ${\displaystyle X=Y}$ почти наверняка, то

    ${\displaystyle M[X]=M[Y]}$.
    

* Математическое ожидание произведения двух независимых или некоррелированных случайных величин ${\displaystyle X,Y}$ равно произведению их математических ожиданий
    
    ${\displaystyle M[XY]=M[X]M[Y]}$.

### Дисперсия
Это мера разброса значений случайной величины относительно её математического ожидания. Обозначается $D[X]$ в русской литературе и $\operatorname {Var}(X)$ (англ. variance) в зарубежной. В статистике часто употребляется обозначение $\sigma _{X}^{2}$ или $\displaystyle \sigma ^{2}$.

Квадратный корень из дисперсии, равный $\displaystyle \sigma$, называется среднеквадратическим отклонением, стандартным отклонением или стандартным разбросом. Стандартное отклонение измеряется в тех же единицах, что и сама случайная величина, а дисперсия измеряется в квадратах этой единицы измерения.

Дисперсией случайной величины называют математическое ожидание квадрата отклонения случайной величины от её математического ожидания.

Пусть $X$ — случайная величина, определённая на некотором вероятностном пространстве. Тогда дисперсией называется ${\displaystyle D[X]=M\left[{\big (}X-M[X]{\big )}^{2}\right]}$

Если случайная величина ${\displaystyle X}$ дискретная, то

${\displaystyle D[X]=\sum _{i=1}^{n}{p_{i}(x_{i}-M[X])^{2}}}$

${\displaystyle D[X]={\frac {1}{2}}\sum _{i=1}^{n}\sum _{j=1}^{n}{p_{i}p_{j}(x_{i}-x_{j})^{2}}=\sum _{i=1}^{n}\sum _{j<i}{p_{i}p_{j}(x_{i}-x_{j})^{2}},}$

__Свойства__

* В силу линейности математического ожидания справедлива формула:

    ${\displaystyle D[X]=M[X^{2}]-\left(M[X]\right)^{2}}$.


* Дисперсия любой случайной величины неотрицательна: ${\displaystyle D[X]\geqslant 0}$.


* Если дисперсия случайной величины конечна, то конечно и её математическое ожидание.


* Если случайная величина равна константе, то её дисперсия равна нулю: ${\displaystyle D[a]=0}$. Верно и обратное: если ${\displaystyle D[X]=0}$, то ${\displaystyle X=M[X]}$ почти всюду.


* Дисперсия суммы двух случайных величин равна:
    ${\displaystyle D[X+Y]=D[X]+D[Y]+2\,{\text{cov}}(X,Y)}$, где ${\displaystyle {\text{cov}}(X,Y)}$ — их ковариация.


* Для дисперсии произвольной линейной комбинации нескольких случайных величин имеет место равенство:
    
    ${\displaystyle D\left[\sum _{i=1}^{n}c_{i}X_{i}\right]=\sum _{i=1}^{n}c_{i}^{2}D[X_{i}]+2\sum _{1\leqslant i<j\leqslant n}c_{i}c_{j}\,{\text{cov}}(X_{i},X_{j})}$, где ${\displaystyle c_{i}\in \mathbb {R} }$.


* В частности, ${\displaystyle D[X_{1}+\ldots +X_{n}]=D[X_{1}]+\ldots +D[X_{n}]}$ для любых независимых или некоррелированных случайных величин, так как их ковариации равны нулю.

* ${\displaystyle D\left[aX\right]=a^{2}D[X]}$.

* ${\displaystyle D\left[-X\right]=D[X]}$.

* ${\displaystyle D\left[X+b\right]=D[X]}$.

### Ковариация

Важной характеристикой совместного распределения двух случайных величин является **ковариация** (или корреляционный момент). Ковариация определяется как математическое ожидание произведения отклонений случайных величин:

${\displaystyle \mathrm {cov} _{XY}=\mathbf {M} \left[(X-\mathbf {M} (X))(Y-\mathbf {M} (Y))\right]=\mathbf {M} (XY)-\mathbf {M} (X)\mathbf {M} (Y)}$.

__Свойства__

* Ковариация двух независимых случайных величин ${\displaystyle \mathbf {X} }$  и ${\displaystyle \mathbf {Y} }$ равна нулю.


* Абсолютная величина ковариации двух случайных величин ${\displaystyle \mathbf {X} }$  и ${\displaystyle \mathbf {Y} }$  не превышает среднего геометрического их дисперсий: ${\displaystyle |\mathrm {cov} _{XY}|\leqslant {\sqrt {\mathrm {D} _{X}\mathrm {D} _{Y}}}}$.


* Ковариация имеет размерность, равную произведению размерности случайных величин, то есть величина ковариации зависит от единиц измерения независимых величин. Данная особенность ковариации затрудняет её использование в целях корреляционного анализа.


### Коэффицент корреляции

Корреляция, или корреляционная зависимость — статистическая взаимосвязь двух или более случайных величин (либо величин, которые можно с некоторой допустимой степенью точности считать таковыми).

Математической мерой корреляции двух случайных величин служит корреляционное отношение ${\displaystyle \mathbf {\eta }}$ либо коэффициент корреляции ${\displaystyle \mathbf {R} }$ (или ${\displaystyle \mathbf {r}}$ ). В случае если изменение одной случайной величины не ведёт к закономерному изменению другой случайной величины, но приводит к изменению другой статистической характеристики данной случайной величины, то подобная связь не считается корреляционной, хотя и является статистической.


__Линейный коэффициент корреляции__

Для устранения недостатка ковариации был введён линейный коэффициент корреляции (или коэффициент корреляции Пирсона). Коэффициент корреляции рассчитывается по формуле:

${\displaystyle \mathbf {r} _{XY}={\frac {\mathbf {cov} _{XY}}{\mathbf {\sigma } _{X}{\sigma }_{Y}}}={\frac {\sum (X-{\bar {X}})(Y-{\bar {Y}})}{\sqrt {\sum (X-{\bar {X}})^{2}\sum (Y-{\bar {Y}})^{2}}}}}$,

где ${\displaystyle {\overline {X}}={\frac {1}{n}}\sum _{t=1}^{n}X_{t}}$, ${\displaystyle {\overline {Y}}={\frac {1}{n}}\sum _{t=1}^{n}Y_{t}}$ — среднее значение выборок.

__Свойства__

* Коэффициент корреляции изменяется в пределах от минус единицы до плюс единицы.


* Неравенство Коши — Буняковского:
    если принять в качестве скалярного произведения двух случайных величин ковариацию ${\displaystyle \langle X,Y\rangle =\mathrm {cov} (X,Y)}$, то норма случайной величины будет равна ${\displaystyle \|X\|={\sqrt {\mathrm {D} [X]}}}$, и следствием неравенства Коши — Буняковского будет:
    
    ${\displaystyle -1\leqslant \mathbb {R} _{X,Y}\leqslant 1}$.


* Коэффициент корреляции равен ${\displaystyle \pm 1}$ тогда и только тогда, когда ${\displaystyle X}$ и ${\displaystyle Y}$ линейно зависимы (исключая события нулевой вероятности, когда несколько точек «выбиваются» из прямой, отражающей линейную зависимость случайных величин):

   ${\displaystyle \mathbb {R} _{X,Y}=\pm 1\Leftrightarrow Y=kX+b,k\neq 0}$,
    где ${\displaystyle k,b\in \mathbb {R} }$. Более того в этом случае знаки ${\displaystyle \mathbb {R} _{X,Y}}$ и ${\displaystyle k}$ совпадают:
    ${\displaystyle \operatorname {sgn} \mathbb {R} _{X,Y}=\operatorname {sgn} k}$.


* Если ${\displaystyle X,Y}$ независимые случайные величины, то ${\displaystyle \mathbb {R} _{X,Y}=0}$. Обратное в общем случае неверно.

__Объяснение на пальцах__ 

| Значение коэффициента | Какая корреляция? | О чем это говорит? |
| --- | --- | -- | 
| r=1 | Сильная положительная корреляция | Люди, которые едят чернику, обладают острым зрением. Ешьте чернику! |
| r<0,5 | Слабая положительная корреляция | Некоторые люди, которые любят чернику, обладают острым зрением. Но это не точно. Короче, ничего не понятно пока. Но лучше есть чернику на всякий случай. |
| r=0 | Корреляция отсутствует | Черника и зрение никак не связаны. |
| r<-0,5 | Слабая отрицательная корреляция | Бывают случаи ухудшения зрения из-за черники. Не стоит рисковать. |
| r=-1 | Сильная отрицательная корреляция | Практически все, кто ел чернику, ослепли. Берегитесь черники! | 

### Сравнение гистограмм

Безусловно, что пользы от гистограмм не будет никакой, если не сравнивать их с эталонными диаграммами. Предположим, есть эталонное изображение $-$ какой-то объект заданных размеров. Также есть множество неизвестных изображений, на которых нужно найти эталонное изображение. Для этого нужно перебирать участки изображений, сравнивая содержимое с эталоном. Можно сравнивать каждую точку из участка, но это будет медленно. Гораздо быстрее по ресурсам $-$ это сравнить гистограммы яркости. Для сравнения гистограмм в OpenCV предусмотрена функция ```cv2.CompareHist()```.

Мы рассмотрим только несколько метрик для сравнения: 

1. __Correlation__ (```method=cv2.HISTCMP_CORREL```)

$d(H_1,H_2) =  \frac{\sum_I (H_1(I) - \bar{H_1}) (H_2(I) - \bar{H_2})}{\sqrt{\sum_I(H_1(I) - \bar{H_1})^2 \sum_I(H_2(I) - \bar{H_2})^2}}$, где $\bar{H_k} =  \frac{1}{N} \sum _J H_k(J)$

2. __Chi-Square__ (```method=cv2.HISTCMP_CHISQR```)

$d(H_1,H_2) =  \sum _I  \frac{\left(H_1(I)-H_2(I)\right)^2}{H_1(I)}$

3. __Intersection__ (```method=cv2.HISTCMP_INTERSECT```)

$d(H_1,H_2) =  1 - \sum _I  \min (H_1(I), H_2(I))$

4. __Bhattacharyya distance__ (```method=cv2.HISTCMP_BHATTACHARYYA```).

$d(H_1,H_2) =  \sqrt{1 - \frac{1}{\sqrt{\bar{H_1} \bar{H_2} N^2}} \sum_I \sqrt{H_1(I) \cdot H_2(I)}}$

In [ ]:
carpet = cv2.imread('img/carpet_ex.jpg')
carpet_gray = cv2.cvtColor(carpet, cv2.COLOR_BGR2GRAY)
forest = cv2.imread('img/forest_ex.jpg')
forest_gray = cv2.cvtColor(forest, cv2.COLOR_BGR2GRAY)

hist_carpet = cv2.calcHist([carpet_gray.ravel()], [0], None, [256], [0,256])
hist_forest = cv2.calcHist([forest_gray.ravel()], [0], None, [256], [0,256])

In [ ]:
fig, m_axs = plt.subplots(1, 2, figsize=(12,8))
ax1, ax2 = m_axs
ax1.set_title('Исходное изображение')
ax1.imshow(carpet_gray, cmap='gray')
ax2.set_title('Выравненное изображение')
ax2.imshow(forest_gray, cmap='gray');

In [ ]:
# возьмем выравненные гистограммы освещенности и сравним через корреляцию

cor_1 = cv2.compareHist(hist_carpet, hist_forest, cv2.HISTCMP_CORREL)

cor_2 = cv2.compareHist(hist_carpet, hist_forest, cv2.HISTCMP_INTERSECT) / np.sum(hist_carpet)

cor_3 = cv2.compareHist(hist_carpet, hist_forest, cv2.HISTCMP_BHATTACHARYYA)

fig, m_axs = plt.subplots(1, 1, figsize=(20, 5))
ax1 = m_axs

ax1.plot(hist_carpet, color='r')
ax1.plot(hist_forest, color='gray')

print(f'Correlation: {cor_1:.3f}\n'
      f'Intersection: {cor_2:.3f}\n'
      f'Bhattacharyya: {cor_3:.3f}')

Заметьте, что в качестве примера были выбраны близкие гистограммы, по сути одна, но с разными операциями выравнивания. Вариативность применения коэффициента корреляции зависит от вашей фантазии и возможности интерпретировать результат.

**Помните**, что корреляция показывает только **линейную** зависимость двух случайных величин. Порой связь может быть гораздо сложнее.

## Практика

1. Выделите на изображении леса, которые мы использовали выше, дерево и небо.

2. Попробуйте разделить два объекта разного класса между собой с помощью сравнения гистограмм.

In [ ]:
# Ваш код

Вообще говоря, сравнивать мы можем любые гистограммы. 

Разберём на примере самый простой случай классификации, когда пространство признака одномерное, а нам нужно разделить 2 класса. Ситуация встречается чаще, чем может представиться: например, когда нужно отличить два сигнала, или сравнить паттерн с образцом. Пусть у нас есть обучающая выборка. При этом получается изображение, где по оси X будет мера похожести, а по оси Y -количество событий с такой мерой. Когда искомый объект похож на себя — получается левая гауссиана. Когда не похож — правая. Значение X=0.4 разделяет выборки так, что ошибочное решение минимизирует вероятность принятия любого неправильного решения. Именно поиском такого разделителя и является задача классификации.

<img src="https://i.ibb.co/gdG3jGj/hist_comp.png" alt="Drawing" style="width: 500px;"/>

### Контрольные вопросы:

1. Как найти оптимальные параметры для clahe (поварируйте параметры)?

2. Как коррелируют гистограммы clahe и первичная?

3. Что будет с нормальной фоткой после clahe?

4. Как сделать __CLAHE__ к цветному изображению?

5. Что будет, если __CLAHE__ применить к другим формата кодирования цвета: HSV, LAB? Какой формат кодирования лучше подходит для работы с коррекцией цвета?

6. Что будет, если __equalization__ применить к другим формата кодирования цвета: HSV, LAB? Какой формат кодирования лучше подходит для работы с коррекцией цвета?

# Матрицы смежности. Gray-Level Co-Occurrence Matrix (GLCM)
 
Давайте искать неоднородности и разделять текстуры!

<img src="https://i.ibb.co/y6FMPLX/image--022.jpg" alt="Drawing" style="width: 400px;"/> 

Матрица совпадений – это статистический метод исследования текстуры изображения в градациях серого. Рассмотрии $I(k,k)$ изображение в градациях серого относительно центрального пикселя $(n_c, m_c)$. Значения совпадения определяется как распределение значений совпадений на данном расстоянии от определенного пикселя $(n_c, m_c)$. Для изображения $I(k,k)$ матрица совпадений $C_M=C_{(D_x, D_y)}(k,k) $определяется как:

$C_M = \sum_{k}^{n=1}\sum_{k}^{m=1} \begin{cases} 1, if \ I(n,m)=k \ and \ I(n+D_x, m+D_y)=k \\ 0, \ otherwise \end{cases}$

где $D_x$, $D_y$ параметр сдвига, задающий взаимное расположение пикселей и определятся следующим образом:

$D_x = D cos(\theta)$, $D_y = D sin(\theta)$, а $\theta$ – смещение, которое определяет направление матрицы от центрального пикселя и расстояние от центрального пикселя, как показано на рисунке

$I(n,m)$ – уровень яркости пикселя изображения, расположенного в точке $(n, m)$.

<img src="img/co-matrix_expl.jpg" alt="Drawing" style="width: 400px;"/> 

## Матрицы смежности: характиристики

1. $Contrast = \sum_{i,j=0}^{levels-1} C_{i,j}(i-j)^2$

Контраст является мерой локального изменения интенсивности, отдавая предпочтение значениям от диагонали (i = j). Чем больше значение, тем больше различие в значениях интенсивности среди соседних вокселей.

2. $Dissimilarity = \sum_{i,j=0}^{levels-1}C_{i,j}|i-j|$

Отражает меру несимметричности элементов в GLCM

3. $Homogeneity = \sum_{i,j=0}^{levels-1}\frac{C_{i,j}}{1+(i-j)^2}$

Отражает близость распредления элементов в GLCM

4. $Energy = \sum_{i,j=0}^{levels-1} C_{i,j}^2$

Энергия является мерой однородных структур на изображении. Большая энергия подразумевает, что в изображении появляется больше пар значений интенсивности, которые соседствуют друг с другом на более высоких частотах.

5. $Aautocorrelation = \sum_{i,j=0}^{levels-1} C_{i,j}ij$

Автокорреляция является мерой величины тонкости и грубости текстуры.

6. $Entropy = - \sum_{i,j=0}^{levels-1}C_{i,j}log_{2}C_{i,j}$ 

Совместная энтропия - это мера случайности / изменчивости значений интенсивности соседства.

7. $Difference \ Entropy = \sum_{k=0}^{levels-2} C_{x-y}(k)log_{2}(C_{x-y}(k) + \epsilon$

Разностная Энтропия является мерой случайности / изменчивости в разнице значений интенсивности окрестности.

8. $Correlation = \sum_{i,j=0}^{levels-1} C_{i,j}\left[\frac{(i-\mu_i) \
(j-\mu_j)}{\sqrt{(\sigma_i^2)(\sigma_j^2)}}\right]$, где

    $\mu_i = \sum_{i=1}^{L}\sum_{j=1}^{L}iC_M$ 

    $\mu_j = \sum_{i=1}^{L}\sum_{j=1}^{L}jC_M$, 

    $\sigma_{i}^2 = \sum_{i=1}^{L}\sum_{j=1}^{L}C_M(i-\mu_{i}^2)$

    $\sigma_{j}^2 = \sum_{i=1}^{L}\sum_{j=1}^{L}C_M(j-\mu_{j}^2)$
    
    Корреляция - это значение между 0 (некоррелированный) и 1 (идеально коррелированный), показывающее линейную зависимость значений уровня серого от их соответствующих пикселей в GLCM.
    
9. $Cluster \ Prominence = \sum_{i,j=0}^{levels-1} (i + j - \mu_{x} - \mu_{y})^4 C_{i,j}$

Распространенность кластеров - это мера асимметрии и асимметрии GLCM. Более высокие значения подразумевают большую асимметрию относительно среднего, в то время как более низкое значение указывает на пик около среднего значения и меньшую вариацию относительно среднего.

10. $Cluster \ Shade = \sum_{i,j=0}^{levels-1} (i + j - \mu_{x} - \mu_{y})^3 C_{i,j}$

Кластерный оттенок - это показатель асимметрии и однородности GLCM. Более высокий оттенок кластера подразумевает большую асимметрию относительно среднего значения.

11. $Cluster \ Tendency = \sum_{i,j=0}^{levels-1} (i + j - \mu_{x} - \mu_{y})^2 C_{i,j} $

Кластерная Тенденция - это мера группирования вокселей с одинаковыми значениями уровня серого.

12. $Joint \ Average = \mu_{x} = \sum_{i,j=0}^{levels-1} C_{i,j}i$

Совместное Среднее - средняя интенсивность уровня серого распределения i.

13. $Difference \ Average = \sum_{k=0}^{levels-2} kC_{x-y}(k)$

$ C_{x-y}(k) = \sum_{i,j=0}^{levels-1} C_{i,j}$, где $|i-j| = k$ и $k = 0, 1, ..., levels - 2$.

Среднее различие измеряет взаимосвязь между появлением пар с одинаковыми значениями интенсивности и появлением пар с различными значениями интенсивности.

14. $Difference \ Variance = \sum_{k=0}^{levels-2} (k-DA)^2 C_{x-y}(k)$

Разница является мерой неоднородности, которая помещает более высокие веса в пары различных уровней интенсивности, которые больше отклоняются от среднего.


## Практика 

1. Построить функцию для расчета матрицы смежности
        На вход: матрица изображения, углы для расчета
        На выходе: матрица смежности
    Можно пользоваться ```from skimage.feature import greycomatrix```
    
    
2. Реализовать 4 характеристики матрицы смежности (на свое усмотрение какие)

3. Как себя ведут характеристики матрциы смежности на примерах ниже? Что по ним можно определить?

4. Определить, где на изображении есть неравномерная структура. Примеры изображений загружены в ячейке ниже.

In [ ]:
carpet = cv2.imread('img/carpet_ex.jpg')
carpet = cv2.cvtColor(carpet, cv2.COLOR_BGR2RGB)
forest = cv2.imread('img/forest_ex.jpg')
forest = cv2.cvtColor(forest, cv2.COLOR_BGR2RGB)
imgs = [carpet, forest]


fig, m_axs = plt.subplots(1, 2, figsize=(12, 9))
m_axs = m_axs.ravel(order='C')
for i, ax in enumerate(m_axs):
    ax.imshow(imgs[i])

In [ ]:
from skimage.feature import greycomatrix

gray_carpet = cv2.cvtColor(carpet, cv2.COLOR_RGB2GRAY)
glcm = greycomatrix(gray_carpet, [2, 16], [0, np.pi/2],
                    normed=False, symmetric=False, levels=256)

In [ ]:
gray_carpet[:4, :4]

In [ ]:
glcm[:4, :4, 0, 0]

In [ ]:
glcm[:4, :4, 1, 0]

In [ ]:
# Ваш Код